## 기본 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Thu Dec 15 06:46:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    52W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !unzip "/content/drive/MyDrive/augda.zip" -d "/content/drive/MyDrive/augda"

In [3]:
import os
os.chdir("/content/drive/MyDrive")

## 모델 기본 세팅

In [4]:
from deeplab2.model.pixel_encoder import moat
import tensorflow as tf
import numpy as np

from tensorflow.python.ops.clip_ops import clip_by_global_norm
!pip install adabelief_tf
from tensorflow.keras.layers import (Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten)
from tensorflow import keras
from adabelief_tf import AdaBeliefOptimizer
opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.01, epsilon=1e-7)

from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

print(tf.test.gpu_device_name())

K.clear_session()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

In [5]:
# 초기 변수 설정
n_classes = 37

### 이미지 크기 설정
img_width, img_height = 500, 500

train_data_dir = '/content/drive/MyDrive/augda/train'
train1_data_dir = '/content/drive/MyDrive/augda/train1'
train2_data_dir = '/content/drive/MyDrive/augda/train2'
validation_data_dir = '/content/drive/MyDrive/augda/validation'
validation_data2_dir = '/content/drive/MyDrive/augda/validation'
test_data_dir = '/content/drive/MyDrive/augda/test'

nb_train_samples =  2576*4
nb_train1_samples =  1282*4
nb_train2_samples =  1282*4
nb_validation_samples = 1104
nb_validation2_samples = 1104
nb_test_samples = 3668
batch_size = 16

In [6]:
# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train3_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')

train1_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   

# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)
train2_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

val_datagen = ImageDataGenerator(
    rescale=1./255,)
val_datagen2 = ImageDataGenerator(
    rescale=1./255,)
test_datagen = ImageDataGenerator(
    rescale=1./255,)


In [7]:
# flow_from_directory : Numpy Array Iterator 객체 생성
# 인자로 설정해주는 directory의 바로 하위 디렉토리 이름을 레이블이라고 간주, 그 디렉토리 아래의 파일들을 해당 레이블의 이미지들이라고 알아서 추측

import tensorflow as tf
import numpy as np
import cv2

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

train3_generator = train3_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

train1_generator = train1_datagen.flow_from_directory(
    train1_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


train2_generator = train2_datagen.flow_from_directory(
    train2_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')
    
  
validation2_generator = val_datagen.flow_from_directory(
    validation_data2_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode=None)


Found 10304 images belonging to 37 classes.
Found 10304 images belonging to 37 classes.
Found 5128 images belonging to 37 classes.
Found 5128 images belonging to 37 classes.
Found 1104 images belonging to 37 classes.
Found 1104 images belonging to 37 classes.
Found 3668 images belonging to 37 classes.


## Train set 1

In [ ]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[500,500,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")
o1 = moat1.output["stage5"]

r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
r1= tf.keras.layers.BatchNormalization()(r1)
out1 = tf.keras.layers.Dense(
    units=37, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r1)  

def create_model():
  return tf.keras.models.Model(moat1.input, out1)

with tf.device('/device:GPU:0'):
  model1=create_model()
  # moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")
  opt2 = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)
  # opt2 = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.01, epsilon=1e-7)
  model1.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='pet_500_disjoint1_augment.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('pet_500_disjoint1_augment.log')
  history = model1.fit_generator(train1_generator,
                    steps_per_epoch = nb_train1_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=40,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  

#########################################################
moat1.save_weights('moat_pet_500_disjoint1_augment.h5')

KeyboardInterrupt: ignored

In [ ]:
moat1.save_weights('moat_pet_500_disjoint1_augment.h5')

## Train set 2

In [ ]:
moat2 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[500,500,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o2 = moat2.output["stage5"]
r2 = tf.keras.layers.Flatten(name='flatten_layer1')(o2)
r2= tf.keras.layers.BatchNormalization()(r2)
out2 = tf.keras.layers.Dense(
    units=37, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r2)

def create_model2():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  model2 = create_model2()
  opt2 = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)
  model2.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='pet_500_disjoint2_augment.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('pet_500_disjoint2_augment.log')
  history = model2.fit_generator(train2_generator,
                    steps_per_epoch = nb_train2_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=40,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  
#########################################################
moat2.save_weights('moat_pet_500_disjoint2_augment.h5')

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



<ipython-input-20-9b90509d35c9>:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model2.fit_generator(train2_generator,


Epoch 1/40
160/160 [==============================] - ETA: 0s - loss: 3.9908 - sparse_categorical_accuracy: 0.0697
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.02849, saving model to pet_500_disjoint2_augment.hdf5
160/160 [==============================] - 342s 2s/step - loss: 3.9908 - sparse_categorical_accuracy: 0.0697 - val_loss: 5.8910 - val_sparse_categorical_accuracy: 0.0285
Epoch 2/40
160/160 [==============================] - ETA: 0s - loss: 2.4629 - sparse_categorical_accuracy: 0.3277
Epoch 2: val_sparse_categorical_accuracy did not improve from 0.02849
160/160 [==============================] - 279s 2s/step - loss: 2.4629 - sparse_categorical_accuracy: 0.3277 - val_loss: 5.4635 - val_sparse_categorical_accuracy: 0.0285
Epoch 3/40
160/160 [==============================] - ETA: 0s - loss: 1.3603 - sparse_categorical_accuracy: 0.5979
Epoch 3: val_sparse_categorical_accuracy improved from 0.02849 to 0.08088, saving model to pet_500_disjoint2_augment.hdf5
160/

## Ensemble section

In [ ]:
first_decay_steps = 1000
initial_learning_rate = 1e-5
lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(    
      initial_learning_rate,
      first_decay_steps))

opt2 = AdaBeliefOptimizer(learning_rate=lr_decayed_fn, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [ ]:
moat3 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[500,500,3])
moat4 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[500,500,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")


moat3.load_weights('moat_pet_500_disjoint1_augment.h5')
moat4.load_weights('moat_pet_500_disjoint2_augment.h5')

moat3.trainable=False
moat4.trainable=False

en_input = Input(shape=(500,500,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)
z= tf.keras.layers.BatchNormalization()(z)

en_out = Dense(units=37, activation='softmax', kernel_initializer="he_normal")(z)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

with tf.device('/device:GPU:0'):
  en_model = create_model3()
  en_model.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='ensemble_augment_1.hdf5', verbose=1, save_best_only=True)
  csv_logger = CSVLogger('ensemble_augment_1.log')
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])


### Ensemble finetuning

In [8]:
first_decay_steps = 1000
initial_learning_rate = 5e-6
lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(    
      initial_learning_rate,
      first_decay_steps))

opt3 = AdaBeliefOptimizer(learning_rate=lr_decayed_fn, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [ ]:
moat3 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[500,500,3])
moat4 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[500,500,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")

en_input = Input(shape=(500,500,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)
z= tf.keras.layers.BatchNormalization()(z)

en_out = Dense(units=37, activation='softmax', kernel_initializer="he_normal")(z)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

en_model2 = create_model3()

moat3.trainable=False
moat4.trainable=False

en_model2.load_weights('ensemble_augment_1.hdf5')

moat3.trainable=True
moat4.trainable=True

with tf.device('/device:GPU:0'):
  en_model2.compile(optimizer=opt3,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  history = en_model2.fit_generator(train3_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10, restore_best_weights=True)])
  
moat3.trainable=False
moat4.trainable=False

en_model2.save_weights('ensemble_augment_1_finetune.h5')

<ipython-input-9-c4c97a527752>:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model2.fit_generator(train3_generator,


Epoch 1/30
 84/644 [==>...........................] - ETA: 2:00:35 - loss: 0.1622 - sparse_categorical_accuracy: 0.9539

## Model assessment

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

batch_size = 16
num_of_test_samples = nb_test_samples

with tf.device('/device:GPU:0'):
  predictions = en_model2.predict_generator(test_generator)

y_pred = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

print(class_labels)
print(confusion_matrix(test_generator.classes, y_pred))
report = classification_report(true_classes, y_pred, target_names=class_labels, digits=5)
print(report)